<a href="https://colab.research.google.com/github/Jair2472/Modelos_de_Prediccion/blob/main/Entrenamietno_ModeloRidge.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# -----------------------------------------------------------------------------
# ARCHIVO: 3_entrenamiento_modelos.py
# ----------------------------------------------------------------------------
# --- Paso 1: Importar Librerías y Cargar Datos Procesados ---
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, r2_score
import xgboost as xgb
import joblib # Para guardar nuestros modelos entrenados
import os

print("Iniciando el script de entrenamiento de modelos...")

# Creamos una carpeta para guardar los modelos si no existe
if not os.path.exists('modelos_entrenados'):
    os.makedirs('modelos_entrenados')

# --- Paso 2: Cargar Datos Procesados ---
# Fundamental: Cargamos `dataset_procesado.csv`
try:
    df = pd.read_csv('dataset_procesado.csv')
    print("Dataset procesado cargado exitosamente.")
except FileNotFoundError:
    print("Error: `dataset_procesado.csv` no encontrado. Ejecuta primero el script `1_preparacion_datos.py`.")
    exit()

# --- Paso 3: Preparación Final de los Datos para el Modelo ---
# Separamos nuestras variables en `X` (las características) e `y` (la variable objetivo).
# Luego, dividimos el dataset en conjuntos de entrenamiento y prueba.

# 'y' es nuestra variable objetivo.
y = df['MargenFinal%']
# 'X' son todas las demás columnas que usaremos como predictores.
X = df.drop(columns=['MargenFinal%', 'Date'])

# Dividimos los datos: 80% para entrenar, 20% para probar.
# 'random_state=42' asegura que la división sea siempre la misma, para poder reproducir los resultados.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Escalado de características: Es una buena práctica escalar las variables numéricas.
scaler = StandardScaler()
numeric_features = ['VentasAcum', 'CostosAcum', 'IngresosAcum', 'DiaDelPeriodo']
X_train[numeric_features] = scaler.fit_transform(X_train[numeric_features])
X_test[numeric_features] = scaler.transform(X_test[numeric_features])
# Guardamos el scaler para poder usarlo en el futuro en datos nuevos
joblib.dump(scaler, 'modelos_entrenados/scaler.pkl')

print(f"Datos listos para entrenar: {X_train.shape[0]} filas de entrenamiento, {X_test.shape[0]} filas de prueba.")

# --- Precisión y Puntos Base (pb) ---
# El objetivo es una precisión de +/- 20 pb (general) y +/- 30 pb (departamental).
# 1 pb = 0.01% = 0.0001  |  20 pb = 0.0020  |  30 pb = 0.0030
# La métrica que mide este error es el Error Absoluto Medio (MAE).
# Buscamos un MAE lo más bajo posible.

# --- Modelo 1: Regresión Lineal (Ridge) - El Baseline ---
print("\n--- Entrenando Modelo 1: Baseline (Ridge) ---")
ridge_model = Ridge(alpha=1.0)
ridge_model.fit(X_train, y_train)
y_pred_ridge = ridge_model.predict(X_test)
mae_ridge = mean_absolute_error(y_test, y_pred_ridge)
r2_ridge = r2_score(y_test, y_pred_ridge)
print(f"MAE General: {mae_ridge:.4f} ({mae_ridge*10000:.0f} pb)")
print(f"R²: {r2_ridge:.4f}")
joblib.dump(ridge_model, 'modelos_entrenados/modelo_ridge.pkl')
print("Modelo Ridge guardado.")

# --- Modelo 2: XGBoost - El Modelo de Alto Rendimiento ---
print("\n--- Entrenando Modelo 2: Alto Rendimiento (XGBoost) ---")
xgb_model = xgb.XGBRegressor(objective='reg:squarederror', n_estimators=100, learning_rate=0.1, max_depth=5, random_state=42)
xgb_model.fit(X_train, y_train)
y_pred_xgb = xgb_model.predict(X_test)
mae_xgb = mean_absolute_error(y_test, y_pred_xgb)
r2_xgb = r2_score(y_test, y_pred_xgb)
print(f"MAE General: {mae_xgb:.4f} ({mae_xgb*10000:.0f} pb)")
print(f"R²: {r2_xgb:.4f}")
joblib.dump(xgb_model, 'modelos_entrenados/modelo_xgboost.pkl')
print("Modelo XGBoost guardado.")


# --- Modelo 3: Estrategia por Departamento (Random Forest) ---
print("\n--- Entrenando Modelo 3: Estrategia por Departamento ---")
modelos_por_depto = {}
predicciones_depto_total = []
y_test_reordenado = []

# Obtenemos la lista de departamentos del dataframe original (antes del one-hot)
deptos_originales = df.filter(regex='^Depto_').columns
deptos_lista = [col.replace('Depto_', '') for col in deptos_originales]

for depto in deptos_lista:
    col_depto = f'Depto_{depto}'

    # Filtrar datos para el departamento actual
    X_train_depto = X_train[X_train[col_depto] == 1]
    y_train_depto = y_train[X_train.index.isin(X_train_depto.index)]
    X_test_depto = X_test[X_test[col_depto] == 1]
    y_test_depto = y_test[X_test.index.isin(X_test_depto.index)]

    if len(X_train_depto) == 0 or len(X_test_depto) == 0:
        continue # Saltar si no hay datos suficientes

    # Entrenar un modelo específico para este departamento
    model_depto = RandomForestRegressor(n_estimators=50, random_state=42, n_jobs=-1)
    model_depto.fit(X_train_depto, y_train_depto)

    # Guardar el modelo entrenado
    joblib.dump(model_depto, f'modelos_entrenados/modelo_depto_{depto}.pkl')

    # Evaluar y almacenar resultados
    y_pred_depto = model_depto.predict(X_test_depto)
    mae_depto = mean_absolute_error(y_test_depto, y_pred_depto)

    predicciones_depto_total.extend(y_pred_depto)
    y_test_reordenado.extend(y_test_depto)

    print(f"  Departamento: {depto}")
    print(f"    MAE: {mae_depto:.4f} ({mae_depto*10000:.0f} pb) | R²: {r2_score(y_test_depto, y_pred_depto):.4f}")

# Evaluar el rendimiento general de la estrategia departamental
mae_total_depto = mean_absolute_error(y_test_reordenado, predicciones_depto_total)
r2_total_depto = r2_score(y_test_reordenado, predicciones_depto_total)

print("\n--- Evaluación General de la Estrategia Departamental ---")
print(f"MAE General: {mae_total_depto:.4f} ({mae_total_depto*10000:.0f} pb)")
print(f"R² General: {r2_total_depto:.4f}")
print("Modelos por departamento guardados.")

print("\n¡Entrenamiento y evaluación de todos los modelos completado!")

Iniciando el script de entrenamiento de modelos...
Dataset procesado cargado exitosamente.
Datos listos para entrenar: 14022 filas de entrenamiento, 3506 filas de prueba.

--- Entrenando Modelo 1: Baseline (Ridge) ---
MAE General: 0.0206 (206 pb)
R²: 0.5610
Modelo Ridge guardado.

--- Entrenando Modelo 2: Alto Rendimiento (XGBoost) ---
MAE General: 0.0124 (124 pb)
R²: 0.8517
Modelo XGBoost guardado.

--- Entrenando Modelo 3: Estrategia por Departamento ---
  Departamento: 123
    MAE: 0.0001 (1 pb) | R²: 0.9996
  Departamento: 223
    MAE: 0.0006 (6 pb) | R²: 0.9957
  Departamento: 276
    MAE: 0.0000 (0 pb) | R²: 1.0000
  Departamento: 323
    MAE: 0.0002 (2 pb) | R²: 0.9992
  Departamento: 371
    MAE: 0.0004 (4 pb) | R²: 0.9993
  Departamento: 429
    MAE: 0.0007 (7 pb) | R²: 0.9822
  Departamento: 453
    MAE: 0.0000 (0 pb) | R²: 1.0000
  Departamento: 541
    MAE: 0.0002 (2 pb) | R²: 0.9995

--- Evaluación General de la Estrategia Departamental ---
MAE General: 0.0003 (3 pb)
R² Ge